In [ ]:
import random
import math
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#通常のMCMC（返り値は[一番最後の点のエネルギー、稲の収量リスト])
def Montecarlo(T, num_sample, MCMC_step=10000):
    new_y = data_amount #以降要素を一つずつ変えていく #data_amountはSeries
    E = [] #Eは配列、どんどん入れていく(一つ後ろで計算の際使用される)
    def Energy(new_y): #
        ener_sum = 0
        for i in range(num_sample-1):
            ener_sum += (2*new_y[i] - 1)*(2*new_y[i+1] - 1) 
            return - ener_sum
    E.append(Energy(new_y)) #第一要素を追加    
    save_y = [] #ほしいベクトルを貯める配列
    count = 0
    while True:
        count += 1
        i = random.randrange(num_sample)
        ori_new_y_i = new_y[i]
        new_y[i] = 1 - new_y[i]
        #new_y[i] = random.uniform(-2, 2)
        selector.fit(data_SNP, new_y)
        mask_trial = selector.get_support()
        E.append(Energy(new_y))  #mask_trialは順番に要素を足していく #E[0]=0とすでに入っている
        r = math.exp(-(E[count]-E[count-1])/T) #温度付き
        R = np.random.uniform(0.0, 1.0)
        if R >= r : #f(z)が一個前のものよりも小さかったら⇔E[count]が一個前のものよりも大きかったら
            new_y[i] = ori_new_y_i #元に戻す   
        if count % MCMC_step == 0:
            save_y.append(new_y)
        else:
            continue
        if len(save_y) >= 1:
            return [E[count], new_y] #P
            break

In [ ]:
def Montecarlo(T, num_sample, MCMC_step):
    new_y=[random.randint(0, 1) for i in range(num_sample)]
    def Energy(new_y): 
        ener_sum = 0
        for i in range(num_sample-1):
            ener_sum += -(2*new_y[i] - 1)*(2*new_y[i+1] - 1) 
        
        ener_sum=ener_sum-(2*new_y[num_sample-1] - 1)*(2*new_y[1] - 1)
        return ener_sum
    E.append(Energy(new_y)) #第一要素を追加    
    save_y[0]=new_y
    #count = 0

    for j in range(MCMC_step):
        #count += 1
        i = random.randrange(num_sample)
        E_before=Energy(new_y)
        ori_new_y_i = new_y[i]
        new_y[i] = 1 - new_y[i]
        E_after=Energy(new_y)
        #new_y[i] = random.uniform(-2, 2)
        #selector.fit(data_SNP, new_y)
        #mask_trial = selector.get_support()
        r = math.exp(-(E_after-E_before)/T) #温度付き
        R = np.random.uniform(0.0, 1.0)
        if R > r : #f(z)が一個前のものよりも小さかったら⇔E[count]が一個前のものよりも大きかったら
            new_y[i] = ori_new_y_i #元に戻す   
    
        save_y[j+1]=new_y
        E.append(Energy(new_y))  #mask_trialは順番に要素を足していく #E[0]=0とすでに入っている
        #if count % MCMC_step == 0:
        #save_y.append(new_y)
        #else:
        #    continue
        #if len(save_y) >= 1:
        #    return [E[count], new_y] #P
        #    break

T=0.1
num_sample=10
MCMC_step=100000

E=[]
save_y=np.empty((MCMC_step+1,num_sample),int) #ほしいベクトルを貯める配列

Montecarlo(T,num_sample,MCMC_step)

print(E)
print(save_y)

fig = plt.figure()
plt.hist(E, bins=20, color='red', alpha=0.6)
plt.savefig("histogram.pdf")

In [1]:
import pandas as pd


# 説明変数（こっちの特徴量上位5つ取り出す）
data_SNP = pd.read_table('training_markers.txt', header=None,  sep =  '\t')

# 目的変数
data_yields = pd.read_table('training_yields.txt', header=None,  sep =  '\t')

data_amount_list = []
for i in range(len(data_yields)):
    data_amount_list.append(data_yields.iloc[i, 0])

data_amount = pd.Series(data_amount_list)

num_sample = data_SNP.shape[0]
#print(num_sample)

from sklearn.feature_selection import SelectKBest, f_regression

# 5つの特徴量を選択
selector = SelectKBest(score_func=f_regression, k=5) 
selector.fit(data_SNP, data_amount)
mask = selector.get_support()    # 各特徴量を選択したか否かのmaskを取得
print(mask)

# 選択した特徴量の列のみ取得
SNP_selected = selector.transform(data_SNP)
print("SNP.shape={}, SNP_selected.shape={}".format(data_SNP.shape, SNP_selected.shape))

[False False False False False False False False False False False False
 False False False  True  True  True  True  True]
SNP.shape=(20, 20), SNP_selected.shape=(20, 5)


/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:300: RuntimeWarning: invalid value encountered in true_divide
  corr /= np.linalg.norm(y)
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [2]:
#import meeting1114_FeatureSelection

(SNPサンプル数, 塩基数)= (20, 20)
[False False False False False False False False False False False False
 False False False  True  True  True  True  True]
SNP.shape=(20, 20), SNP_selected.shape=(20, 5)


/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:300: RuntimeWarning: invalid value encountered in true_divide
  corr /= np.linalg.norm(y)
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [3]:
from sklearn.metrics.cluster import normalized_mutual_info_score

#print(Energy(mask))
#print(mask)
#print(type(E))
#print(E[0])

#mask_trial ...new_yによって選ばれた塩基mask(x)、forループを回すたび値が変わる、maskと比較するためにあるだけなので保存する必要なし

import math
import random

#通常のMCMC（返り値は[一番最後の点のエネルギー、稲の収量リスト])
def Montecarlo(T, num_sample, MCMC_step=10000):
    new_y = data_amount #以降要素を一つずつ変えていく #data_amountはSeries
    E = [] #Eは配列、どんどん入れていく(一つ後ろで計算の際使用される)
    def Energy(new_y): #
        ener_sum = 0
        for i in range(num_sample-1):
            ener_sum += (2*new_y[i] - 1)*(2*new_y[i+1] - 1) 
            return - ener_sum
    E.append(Energy(new_y)) #第一要素を追加    
    save_y = [] #ほしいベクトルを貯める配列
    count = 0
    while True:
        count += 1
        i = random.randrange(num_sample)
        ori_new_y_i = new_y[i]
        new_y[i] = 1 - new_y[i]
        #new_y[i] = random.uniform(-2, 2)
        selector.fit(data_SNP, new_y)
        mask_trial = selector.get_support()
        E.append(Energy(new_y))  #mask_trialは順番に要素を足していく #E[0]=0とすでに入っている
        r = math.exp(-(E[count]-E[count-1])/T) #温度付き
        R = np.random.uniform(0.0, 1.0)
        if R >= r : #f(z)が一個前のものよりも小さかったら⇔E[count]が一個前のものよりも大きかったら
            new_y[i] = ori_new_y_i #元に戻す   
        if count % MCMC_step == 0:
            save_y.append(new_y)
        else:
            continue
        if len(save_y) >= 1:
            return [E[count], new_y] #P
            break

In [4]:
# レプリカモンテカルロ法（返り値は[要素K個のエネルギーリスト、]
def Replica(Ts, num_sample, num_step=2000): #Tsは後で指定
    E_replica = [] #要素K個のエネルギーリスト
    T_replica = [] #要素K個の温度リスト
    count_replica = 0
    while True:
        for l in Ts:
            Montecarlo(l, num_sample)
            E_replica.append(Montecarlo(l, num_sample)[0])
            T_replica.append(l)
        count_replica += 1
        if count_replica % num_step != 0:
            E_replica = []
            T_replica = []
        else:
            break       
    k = random.randint(1, len(Ts)-1)
    if np.random.uniform(size=1) < math.exp((T_replica[k+1]-T_replica[k])*(E_replica[k+1]-E_replica[k])):
        Ts[k], Ts[k+1] = Ts[k+1], Ts[k]
    return [E_replica, Ts]

In [ ]:
import matplotlib.pyplot as plt
import math 
import numpy as np

num_sample = 20
Ts = range(1, 100)
Ys = [Montecarlo(T, num_sample)[0] for T in Ts]
plt.plot(Ts, Ys)

/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:300: RuntimeWarning: invalid value encountered in true_divide
  corr /= np.linalg.norm(y)
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/loca

In [ ]:
# / feature_selection / univariate_selection.py

# 回帰 f-regression
degrees_of_freedom = y.size - (2 if center else 1)
F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
pv = stats.f.sf(F, 1, degrees_of_freedom)